# Authentication

After you complete this section you will learn how to make your first API request to the APF and how to keep this connection alive:

1. [Registering a user account on the C2](#registering-a-user-account-on-the-C2)
1. [Generating an API Token](#generating-an-API-Token)
1. [Token refresh](#Token-Refresh)

#### Notebook configuration

In [2]:
import pickle
with open('apf_settings.pickle', 'rb') as f:
    API_ROOT,APF,TOKEN,DEPLOYMENT_ID,\
    ALGORITHM_ID,ALGORITHM_VERSION,ORCHESTRATOR_ID,ALGORITHM_INSTANCE_ID = pickle.load(f)

## Registering a user account on the C2

Whilst the algorithms themselves will interact with the system using
_API Tokens_, user(s) from your institution are responsible for generating
these tokens, which will be tied to their user accounts.

If you have not already signed up for a C2 user account, you simply need to
go to:

https://piloting.c2.noc.ac.uk/auth/login

Enter the email address and desired password you want for your account, and
click **Sign Up** instead of Login.

<div class="alert alert-block alert-info">
<b>Note:</b> We are currently manually approving users' access to the C2 so if you are
not already in contact with us please also send an email to c2@noc.ac.uk
detailing what you intend on using the C2 APIs for; please include the email
you used to sign up. We will email you back when your account has been
approved, until then you won't be able to log in.</div>



## Generating an API Token

Once your account is active, you can generate a token by going to:

https://api.c2.noc.ac.uk/charon/tokens/issue

in a web browser -- it will ask you for your login details. Copy the token and
paste it in the cell below.


<div class="alert alert-block alert-danger">
<b>Important:</b> Copy the token before you navigate away from that page as
the page cannot be refreshed. You, the user are responsible for managing these
tokens: revoking them when no longer used. Please keep your tokens safe: don't
store them in plaintext in any code repositories for example.
</div>

You will be able to submit actions to the piloting framework using the
generated token, in an `Authorization: Bearer` header on your web requests.

Throughout the rest of the notebooks, we will be making use of a helper
function `api_request` which wraps the `requests` library. Here an example of
its use:

In [3]:
from c2_lib import init
from c2_lib import api_request
import json

TOKEN = '464107350f054dc599fa5b2550d11d49'                    

response = api_request(APF + '/drivers',method='get',token=TOKEN)
print(json.dumps(response.json(), indent=4, sort_keys=True))

[
    {
        "description": "Simple driver which accepts actions asynchronously from all connected\n    algorithms, and calls the relevant APIs straight away. It does not wait\n    for the associated platform to be in the correct state -- unless\n    the platform, related APIs or the driver itself reach an error/failure\n    state, in which case the driver halts.\n\n    For example: this driver would take accepted plan updates at any time and\n    convert them into platform-specific files, upload and commit them to\n    greatblue2 and send them to the platform instantly, without requiring\n    the platform to have connected and had its state validated.\n\n    However, if an error is then raised somewhere, the driver will process\n    no further actions.",
        "module_name": "simple",
        "parameter_schema": null,
        "status_schema": null
    },
    {
        "description": "Does what `simple` does, but does not upload or send files via the\n    greatblue2 API -- it crea

## Token Refresh

_Currently, tokens remain active 1 hour after their last use, so they don't
need to be refreshed - but we would encourage you to use the token refresh
mechanism now for better security, and it may be required in future._

<div class="alert alert-block alert-info">
<b>Note:</b> As the notebook is interactive, not a long-running process, we
won't be making use of refresh tokens. If your token expires whilst you are
using it, you can just request a new one.</div>

<div class="alert alert-block alert-warning">
<b>Warning:</b> By default, tokens expire after an hour. Before they expire,
you can "refresh" them. This changes the token hash but keeps the permissions.
This is for security, so that any credentials stored in plain text on disk
will only be valid for a short period. This means when developing your
algorithms you want to be having them refresh their token(s) approximately
every 55 minutes.
</div>

For refresh, you could use a thread which simply pings the Automated Piloting
API every 55 minutes, to update the token. Once this thread is `start()`ed,
it will continue to run in the background. Your original token will be
invalidated upon refresh.

Here is some example code which would do this:

In [4]:
from c2_lib import auth
c2_auth = auth.authenticate(api_root=API_ROOT, token=TOKEN, apf=APF)

New token: 91ed8e01cca14407b83940fcc20d7a31


Here you can see the source code:

In [ ]:
# %load c2_lib/auth.py
import threading
import datetime
import logging
import time
from . import api_request

class C2Auth(threading.Thread):
    """
    A thread which, once started, sits in the background and attempts to
    refresh a C2 authentication token against the `charon` API at a regular
    interval (default of 55 minutes).

    The refreshed token is then stored in `.token`, and you can use this
    token when making requests to C2 APIs.
    """

    def __init__(
            self,
            original_token,
            api_root,
            refresh_interval=55 * 60,
            retry_interval=60,
            max_retries=3,
            daemon=True,
            *args,
            **kwargs
    ):
        """
        :param original_token: The original token you received when you
                               visited the /charon/tokens/issue page in a
                               web browser.
        :param api_root: The root path of the C2 API in use
                         (https://api.c2.noc.ac.uk is the default, it could
                          be https://api.test.c2.noc.ac.uk for example)
        :param refresh_interval: Amount of time (in seconds) between
                                 token refresh attempts. Defaults to
                                 55 minutes.
        :param retry_interval: Amount of time to wait (in seconds) between
                               retries, when the first attempt at refreshing
                               a token has failed.
        :param max_retries: Total number of attempts to make at refreshing
                            a token, before giving up
        :param daemon: Tell the Python interpreter that it doesn't have to
                       wait for this thread to finish when quitting
                       (default True)
        """
        super(C2Auth, self).__init__(*args, **kwargs)
        self.original_token = original_token
        self.token = original_token
        self.api_root = api_root
        self.last_used = None
        self.last_response = None
        self.last_error = None
        self.refresh_interval = refresh_interval
        self.retry_interval = retry_interval
        self.refreshed = False
        self.max_retries = max_retries
        self.retries = 0
        self.running = False
        self.daemon = daemon

    def refresh(self):
        url = self.api_root + '/charon/tokens/' + self.token
        try:
            logging.debug('Attempting to refresh token using %s', url)
            response = api_request(
                url=url,
                # Remember: The original (current) token needs to be passed to
                # the refresh call, both in the URL (specifying which token
                # you, the authorized user, want to refresh), and in the
                # "Authorization: Bearer" header (using the token to identify
                # and authorize you to use that API call)
                token=self.token,
                method='put'
            )
            while response.status_code not in [200, 401] \
                    and self.retries < self.max_retries:
                logging.warning(
                    'Token refresh failed for %s, retry %s/%s: %s %s',
                    url, self.retries, self.max_retries,
                    response.status_code, response.content
                )
                time.sleep(self.retry_interval)
                self.refreshed = False
                response = api_request(
                    url=self.api_root + '/tokens/' + self.token,
                    token=self.token,
                    method='put'
                )
                self.retries += 1
            self.last_response = response
            self.last_used = datetime.datetime.utcnow()
            self.refreshed = response.status_code == 200
            if self.refreshed:
                self.token = response.json()['payload']
                self.retries = 0
            logging.debug('Token refreshed: %s', self.refreshed)
        except BaseException as e:
            logging.exception(
                'Token refresh failed whilst trying to access %s', url
            )
            self.last_error = e
            self.refreshed = False

    def run(self):
        self.running = True
        while self.running:
            self.refresh()
            time.sleep(self.refresh_interval)


def authenticate(api_root, token, apf):
    """
    Creates and starts a C2Auth thread.
    """
    if 'localhost' not in apf:
        assert token != '<paste your token from charon here>',\
            "token needs to be set to a valid access token from charon"
    c2_auth = C2Auth(
        original_token=token,
        api_root=api_root
    )
    if 'localhost' not in apf:
        # Note: once this thread starts, the TOKEN defined above will become
        # invalid as the C2 API will refresh it straight away.
        c2_auth.start()
        time.sleep(5)
        print('New token:', c2_auth.token)
    return c2_auth



Finally, you can test the new token valid for another hour:

In [6]:
from c2_lib import api_request

TOKEN = c2_auth.token   # renewed token
response = api_request(API_ROOT+'/apf/drivers',method='get',token=TOKEN)
print(json.dumps(response.json(), indent=4, sort_keys=True))

[
    {
        "description": "Simple driver which accepts actions asynchronously from all connected\n    algorithms, and calls the relevant APIs straight away. It does not wait\n    for the associated platform to be in the correct state -- unless\n    the platform, related APIs or the driver itself reach an error/failure\n    state, in which case the driver halts.\n\n    For example: this driver would take accepted plan updates at any time and\n    convert them into platform-specific files, upload and commit them to\n    greatblue2 and send them to the platform instantly, without requiring\n    the platform to have connected and had its state validated.\n\n    However, if an error is then raised somewhere, the driver will process\n    no further actions.",
        "module_name": "simple",
        "parameter_schema": null,
        "status_schema": null
    },
    {
        "description": "Does what `simple` does, but does not upload or send files via the\n    greatblue2 API -- it crea

Finally save all the variables in the pickle file:

In [7]:
import pickle
with open('apf_settings.pickle', 'w+b') as f:
    pickle.dump([API_ROOT,APF,TOKEN,DEPLOYMENT_ID,\
                 ALGORITHM_ID,ALGORITHM_VERSION,ORCHESTRATOR_ID,ALGORITHM_INSTANCE_ID], f)

### A note about the token refresh thread

If you ran the cell above which calls `auth.authenticate`, this started a
background thread for the periodic token refresh. This will continue to
run for as long as your Jupyter Notebook is running - there is no guarantee
it will stop when you stop or restart the kernel, only stopping the notebook
server itself is guaranteed to end the thread.

You can check whether the token refresh thread is alive (still running,
and still refreshing tokens), with the threading `is_alive` function:

In [8]:
c2_auth.is_alive()

True

You can stop it from refreshing the token any further by setting the
`running` parameter to `False`. **This will not stop the thread immediately**,
but it will end the thread when it is next due to refresh the token.

In [9]:
c2_auth.running = False

# Authorization

The C2 APIs filter results based on your user account and access token's
permissions. If you do not have permission to view a platform or deployment
for which you are requesting data, you will not receive results for that
deployment or platform.

Permissions can be time-based, meaning that you may be given access to a
platform within the duration of a deployment.

When requesting information about a single specific
platform or deployment, you may receive a `403 Forbidden` HTTP response which
will describe which permission(s) you are missing.

However, if you are making a request to an API endpoint which returns a
filterable collection of data (i.e. a list of deployments), and you don't
have permission to view any of the deployments or platforms you are interested
in, you may receive an empty list in response.

As an algorithm developer, currently you will have permissions tied to your
user account, and the C2 Team is responsible for delegating those permissions.

We are planning to further automate this in the future, such your account
will automatically be granted specific permissions to access data for the
instances of your algorithms, and the deployments and platforms for which
those instances are associated to. Once an instance is deleted, or a
deployment ends, you would lose access to that deployment and platform unless
your user has otherwise been granted permission.